In [136]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from workalendar.asia import SouthKorea
cal = SouthKorea()
pd.options.display.max_columns = 50
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import koreanize_matplotlib
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score,train_test_split,KFold
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [137]:
data = pd.read_csv('train.csv')
data['location'] = np.where(data['location']=='J',0,1)
data['timestamp'] = pd.to_datetime(data['timestamp'])

data_TG = data[data['item']=='TG'].copy()
data_BC = data[data['item']=='BC'].copy()
data_RD = data[data['item']=='RD'].copy()
data_CR = data[data['item']=='CR'].copy()
data_CB = data[data['item']=='CB'].copy()

data_TG.loc[:,'year'] = data_TG['timestamp'].dt.year
data_TG.loc[:,'month'] = data_TG['timestamp'].dt.month
data_TG.loc[:,'dayofweek'] = data_TG['timestamp'].dt.dayofweek

data_BC.loc[:,'year'] = data_BC['timestamp'].dt.year
data_BC.loc[:,'month'] = data_BC['timestamp'].dt.month
data_BC.loc[:,'dayofweek'] = data_BC['timestamp'].dt.dayofweek

data_RD.loc[:,'year'] = data_RD['timestamp'].dt.year
data_RD.loc[:,'month'] = data_RD['timestamp'].dt.month
data_RD.loc[:,'dayofweek'] = data_RD['timestamp'].dt.dayofweek

data_CR.loc[:,'year'] = data_CR['timestamp'].dt.year
data_CR.loc[:,'month'] = data_CR['timestamp'].dt.month
data_CR.loc[:,'dayofweek'] = data_CR['timestamp'].dt.dayofweek

data_CB.loc[:,'year'] = data_CB['timestamp'].dt.year
data_CB.loc[:,'month'] = data_CB['timestamp'].dt.month
data_CB.loc[:,'dayofweek'] = data_CB['timestamp'].dt.dayofweek

data_TG = data_TG.drop('item',axis=1)
data_BC = data_BC.drop('item',axis=1)
data_RD = data_RD.drop('item',axis=1)
data_CR = data_CR.drop('item',axis=1)
data_CB = data_CB.drop('item',axis=1)

holiday_2019=pd.DataFrame(cal.holidays(2019))
holiday_2020=pd.DataFrame(cal.holidays(2020))
holiday_2021=pd.DataFrame(cal.holidays(2021))
holiday_2022=pd.DataFrame(cal.holidays(2022))
holiday_2023=pd.DataFrame(cal.holidays(2023))

holiday = pd.concat([holiday_2019,holiday_2020,holiday_2021,holiday_2022,holiday_2023],axis=0)
holiday = holiday.rename(columns={0:'timestamp'})
holiday['timestamp'] = pd.to_datetime(holiday['timestamp'])

data_TG = pd.merge(data_TG,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
data_TG['holiday'] = data_TG['holiday'].fillna(0)
data_TG['holiday'] = np.where(data_TG['holiday']==0,0,1)
data_TG['holiday'] = np.where(data_TG['dayofweek']>=5,1,data_TG['holiday'])
data_TG['season'] = data_TG['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

data_BC = pd.merge(data_BC,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
data_BC['holiday'] = data_BC['holiday'].fillna(0)
data_BC['holiday'] = np.where(data_BC['holiday']==0,0,1)
data_BC['holiday'] = np.where(data_BC['dayofweek']>=5,1,data_BC['holiday'])
data_BC['season'] = data_BC['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

data_RD = pd.merge(data_RD,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
data_RD['holiday'] = data_RD['holiday'].fillna(0)
data_RD['holiday'] = np.where(data_RD['holiday']==0,0,1)
data_RD['holiday'] = np.where(data_RD['dayofweek']>=5,1,data_RD['holiday'])
data_RD['season'] = data_RD['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

data_CR = pd.merge(data_CR,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
data_CR['holiday'] = data_CR['holiday'].fillna(0)
data_CR['holiday'] = np.where(data_CR['holiday']==0,0,1)
data_CR['holiday'] = np.where(data_CR['dayofweek']>=5,1,data_CR['holiday'])
data_CR['season'] = data_CR['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

data_CB = pd.merge(data_CB,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
data_CB['holiday'] = data_CB['holiday'].fillna(0)
data_CB['holiday'] = np.where(data_CB['holiday']==0,0,1)
data_CB['holiday'] = np.where(data_CB['dayofweek']>=5,1,data_CB['holiday'])
data_CB['season'] = data_CB['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

inter = pd.read_csv('international_trade.csv')
inter = inter.drop('무역수지',axis=1)
inter = inter.rename(columns={'기간':'timestamp','수출 중량':'Export supply(kg)','수출 금액':'Export price','수입 중량':'import supply(kg)','수입 금액':'import price'})
inter['timestamp'] = pd.to_datetime(inter['timestamp'])
inter_tg = inter[inter['품목명']=='감귤'].reset_index(drop=True)
inter_bc = inter[inter['품목명']=='꽃양배추와 브로콜리(broccoli)'].reset_index(drop=True)
inter_cb = inter[inter['품목명']=='양배추'].reset_index(drop=True)
inter_cr = inter[inter['품목명']=='당근'].reset_index(drop=True)
inter_tg = inter_tg.drop('품목명',axis=1)
inter_bc = inter_bc.drop('품목명',axis=1)
inter_cb = inter_cb.drop('품목명',axis=1)
inter_cr = inter_cr.drop('품목명',axis=1)

data_TG = pd.merge(data_TG,inter_tg,how='left',on='timestamp')
data_BC = pd.merge(data_BC,inter_bc,how='left',on='timestamp')
data_CR = pd.merge(data_CR,inter_cr,how='left',on='timestamp')
data_CB = pd.merge(data_CB,inter_cb,how='left',on='timestamp')

data_TG['Export supply(kg)'] = data_TG['Export supply(kg)'].fillna(method='ffill')
data_TG['Export price'] = data_TG['Export price'].fillna(method='ffill')
data_TG['import supply(kg)'] = data_TG['import supply(kg)'].fillna(method='ffill')
data_TG['import price'] = data_TG['import price'].fillna(method='ffill')

data_BC['Export supply(kg)'] = data_BC['Export supply(kg)'].fillna(method='ffill')
data_BC['Export price'] = data_BC['Export price'].fillna(method='ffill')
data_BC['import supply(kg)'] = data_BC['import supply(kg)'].fillna(method='ffill')
data_BC['import price'] = data_BC['import price'].fillna(method='ffill')

data_CR['Export supply(kg)'] = data_CR['Export supply(kg)'].fillna(method='ffill')
data_CR['Export price'] = data_CR['Export price'].fillna(method='ffill')
data_CR['import supply(kg)'] = data_CR['import supply(kg)'].fillna(method='ffill')
data_CR['import price'] = data_CR['import price'].fillna(method='ffill')

data_CB['Export supply(kg)'] = data_CB['Export supply(kg)'].fillna(method='ffill')
data_CB['Export price'] = data_CB['Export price'].fillna(method='ffill')
data_CB['import supply(kg)'] = data_CB['import supply(kg)'].fillna(method='ffill')
data_CB['import price'] = data_CB['import price'].fillna(method='ffill')

data_TG.loc[:,'week'] = data_TG['timestamp'].dt.isocalendar().week
data_BC.loc[:,'week'] = data_BC['timestamp'].dt.isocalendar().week
data_RD.loc[:,'week'] = data_RD['timestamp'].dt.isocalendar().week
data_CB.loc[:,'week'] = data_CB['timestamp'].dt.isocalendar().week
data_CR.loc[:,'week'] = data_CR['timestamp'].dt.isocalendar().week

In [138]:
test = pd.read_csv('test.csv')
test['location'] = np.where(test['location']=='J',0,1)
test['timestamp'] = pd.to_datetime(test['timestamp'])

test_TG = test[test['item']=='TG'].copy()
test_BC = test[test['item']=='BC'].copy()
test_RD = test[test['item']=='RD'].copy()
test_CR = test[test['item']=='CR'].copy()
test_CB = test[test['item']=='CB'].copy()

test_TG.loc[:,'year'] = test_TG['timestamp'].dt.year
test_TG.loc[:,'month'] = test_TG['timestamp'].dt.month
test_TG.loc[:,'dayofweek'] = test_TG['timestamp'].dt.dayofweek

test_BC.loc[:,'year'] = test_BC['timestamp'].dt.year
test_BC.loc[:,'month'] = test_BC['timestamp'].dt.month
test_BC.loc[:,'dayofweek'] = test_BC['timestamp'].dt.dayofweek

test_RD.loc[:,'year'] = test_RD['timestamp'].dt.year
test_RD.loc[:,'month'] = test_RD['timestamp'].dt.month
test_RD.loc[:,'dayofweek'] = test_RD['timestamp'].dt.dayofweek

test_CR.loc[:,'year'] = test_CR['timestamp'].dt.year
test_CR.loc[:,'month'] = test_CR['timestamp'].dt.month
test_CR.loc[:,'dayofweek'] = test_CR['timestamp'].dt.dayofweek

test_CB.loc[:,'year'] = test_CB['timestamp'].dt.year
test_CB.loc[:,'month'] = test_CB['timestamp'].dt.month
test_CB.loc[:,'dayofweek'] = test_CB['timestamp'].dt.dayofweek

test_TG = test_TG.drop('item',axis=1)
test_BC = test_BC.drop('item',axis=1)
test_RD = test_RD.drop('item',axis=1)
test_CR = test_CR.drop('item',axis=1)
test_CB = test_CB.drop('item',axis=1)

test_TG = pd.merge(test_TG,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
test_TG['holiday'] = test_TG['holiday'].fillna(0)
test_TG['holiday'] = np.where(test_TG['holiday']==0,0,1)
test_TG['holiday'] = np.where(test_TG['dayofweek']>=5,1,test_TG['holiday'])
test_TG['season'] = test_TG['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

test_BC = pd.merge(test_BC,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
test_BC['holiday'] = test_BC['holiday'].fillna(0)
test_BC['holiday'] = np.where(test_BC['holiday']==0,0,1)
test_BC['holiday'] = np.where(test_BC['dayofweek']>=5,1,test_BC['holiday'])
test_BC['season'] = test_BC['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

test_RD = pd.merge(test_RD,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
test_RD['holiday'] = test_RD['holiday'].fillna(0)
test_RD['holiday'] = np.where(test_RD['holiday']==0,0,1)
test_RD['holiday'] = np.where(test_RD['dayofweek']>=5,1,test_RD['holiday'])
test_RD['season'] = test_RD['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

test_CR = pd.merge(test_CR ,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
test_CR['holiday'] = test_CR['holiday'].fillna(0)
test_CR['holiday'] = np.where(test_CR['holiday']==0,0,1)
test_CR['holiday'] = np.where(test_CR['dayofweek']>=5,1,test_CR['holiday'])
test_CR['season'] = test_CR['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

test_CB = pd.merge(test_CB,holiday,how='left',on='timestamp').rename(columns={1:'holiday'})
test_CB['holiday'] = test_CB['holiday'].fillna(0)
test_CB['holiday'] = np.where(test_CB['holiday']==0,0,1)
test_CB['holiday'] = np.where(test_CB['dayofweek']>=5,1,test_CB['holiday'])
test_CB['season'] = test_CB['month'].map({1:0,2:0,3:1,4:1,5:1,6:2,7:2,8:2,9:3,10:3,11:3,12:3})

inter_tg['year'] = inter_tg['timestamp'].dt.year
inter_bc['year'] = inter_bc['timestamp'].dt.year
inter_cr['year'] = inter_cr['timestamp'].dt.year
inter_cb['year'] = inter_cb['timestamp'].dt.year

inter_tg['month'] = inter_tg['timestamp'].dt.month
inter_bc['month'] = inter_bc['timestamp'].dt.month
inter_cr['month'] = inter_cr['timestamp'].dt.month
inter_cb['month'] = inter_cb['timestamp'].dt.month

inter_tg = inter_tg.drop('timestamp',axis=1)
inter_bc = inter_bc.drop('timestamp',axis=1)
inter_cr = inter_cr.drop('timestamp',axis=1)
inter_cb = inter_cb.drop('timestamp',axis=1)

test_TG['week'] = test_TG['timestamp'].dt.isocalendar().week
test_RD['week'] = test_RD['timestamp'].dt.isocalendar().week
test_BC['week'] = test_BC['timestamp'].dt.isocalendar().week
test_CR['week'] = test_CR['timestamp'].dt.isocalendar().week
test_CB['week'] = test_CB['timestamp'].dt.isocalendar().week

In [139]:
a = data_TG.groupby(['month'])[['Export supply(kg)']].mean().reset_index()
b = data_TG.groupby(['month'])[['Export price']].mean().reset_index()
c = data_TG.groupby(['month'])[['import supply(kg)']].mean().reset_index()
d = data_TG.groupby(['month'])[['import price']].mean().reset_index()

data_TG = data_TG.drop(['Export supply(kg)','Export price','import supply(kg)','import price'],axis=1)

data_TG = pd.merge(data_TG,a,how='left')
data_TG = pd.merge(data_TG,b,how='left')
data_TG = pd.merge(data_TG,c,how='left')
data_TG = pd.merge(data_TG,d,how='left')
test_TG = pd.merge(test_TG,a,how='left')
test_TG = pd.merge(test_TG,b,how='left')
test_TG = pd.merge(test_TG,c,how='left')
test_TG = pd.merge(test_TG,d,how='left')
####
a = data_BC.groupby(['month'])[['Export supply(kg)']].mean().reset_index()
b = data_BC.groupby(['month'])[['Export price']].mean().reset_index()
c = data_BC.groupby(['month'])[['import supply(kg)']].mean().reset_index()
d = data_BC.groupby(['month'])[['import price']].mean().reset_index()

data_BC = data_BC.drop(['Export supply(kg)','Export price','import supply(kg)','import price'],axis=1)

data_BC = pd.merge(data_BC,a,how='left')
data_BC = pd.merge(data_BC,b,how='left')
data_BC = pd.merge(data_BC,c,how='left')
data_BC = pd.merge(data_BC,d,how='left')
test_BC = pd.merge(test_BC,a,how='left')
test_BC = pd.merge(test_BC,b,how='left')
test_BC = pd.merge(test_BC,c,how='left')
test_BC = pd.merge(test_BC,d,how='left')
####
a = data_CR.groupby(['month'])[['Export supply(kg)']].mean().reset_index()
b = data_CR.groupby(['month'])[['Export price']].mean().reset_index()
c = data_CR.groupby(['month'])[['import supply(kg)']].mean().reset_index()
d = data_CR.groupby(['month'])[['import price']].mean().reset_index()

data_CR = data_CR.drop(['Export supply(kg)','Export price','import supply(kg)','import price'],axis=1)

data_CR = pd.merge(data_CR,a,how='left')
data_CR = pd.merge(data_CR,b,how='left')
data_CR = pd.merge(data_CR,c,how='left')
data_CR = pd.merge(data_CR,d,how='left')
test_CR = pd.merge(test_CR,a,how='left')
test_CR = pd.merge(test_CR,b,how='left')
test_CR = pd.merge(test_CR,c,how='left')
test_CR = pd.merge(test_CR,d,how='left')
####
a = data_CB.groupby(['month'])[['Export supply(kg)']].mean().reset_index()
b = data_CB.groupby(['month'])[['Export price']].mean().reset_index()
c = data_CB.groupby(['month'])[['import supply(kg)']].mean().reset_index()
d = data_CB.groupby(['month'])[['import price']].mean().reset_index()

data_CB = data_CB.drop(['Export supply(kg)','Export price','import supply(kg)','import price'],axis=1)

data_CB = pd.merge(data_CB,a,how='left')
data_CB = pd.merge(data_CB,b,how='left')
data_CB = pd.merge(data_CB,c,how='left')
data_CB = pd.merge(data_CB,d,how='left')
test_CB = pd.merge(test_CB,a,how='left')
test_CB = pd.merge(test_CB,b,how='left')
test_CB = pd.merge(test_CB,c,how='left')
test_CB = pd.merge(test_CB,d,how='left')

In [140]:
a = data_TG.groupby(['corporation','month'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'month_price'})
b = data_TG.groupby(['corporation','week'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'week_price'})
c = data_TG.groupby(['corporation','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price'})
# d = data_TG.groupby(['corporation','year','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price_year'})
# e = data_TG.groupby(['corporation','season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'season_price'})
# f = data_TG.groupby(['season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'t_season_price'})

g = data_TG.groupby(['corporation','month'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'month_supply'})
h = data_TG.groupby(['corporation','season'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'season_supply'})
i = data_TG.groupby(['corporation','dayofweek'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'dayofweek_supply'})

data_TG = pd.merge(data_TG,a,how='left')
data_TG = pd.merge(data_TG,b,how='left')
data_TG = pd.merge(data_TG,c,how='left')
# data_TG = pd.merge(data_TG,d,how='left')
# data_TG = pd.merge(data_TG,e,how='left')
# data_TG = pd.merge(data_TG,f,how='left')
data_TG = pd.merge(data_TG,g,how='left')
data_TG = pd.merge(data_TG,h,how='left')
data_TG = pd.merge(data_TG,i,how='left')


test_TG = pd.merge(test_TG,a,how='left')
test_TG = pd.merge(test_TG,b,how='left')
test_TG = pd.merge(test_TG,c,how='left')
# test_TG = pd.merge(test_TG,d,how='left')
# test_TG = pd.merge(test_TG,e,how='left')
# test_TG = pd.merge(test_TG,f,how='left')
test_TG = pd.merge(test_TG,g,how='left')
test_TG = pd.merge(test_TG,h,how='left')
test_TG = pd.merge(test_TG,i,how='left')



a = data_BC.groupby(['corporation','month'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'month_price'})
b = data_BC.groupby(['corporation','week'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'week_price'})
c = data_BC.groupby(['corporation','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price'})
# d = data_BC.groupby(['corporation','year','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price_year'})
# e = data_BC.groupby(['corporation','season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'season_price'})
# f = data_BC.groupby(['season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'t_season_price'})

g = data_BC.groupby(['corporation','month'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'month_supply'})
h = data_BC.groupby(['corporation','season'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'season_supply'})
i = data_BC.groupby(['corporation','dayofweek'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'dayofweek_supply'})



data_BC = pd.merge(data_BC,a,how='left')
data_BC = pd.merge(data_BC,b,how='left')
data_BC = pd.merge(data_BC,c,how='left')
# data_BC = pd.merge(data_BC,d,how='left')
# data_BC = pd.merge(data_BC,e,how='left')
# data_BC = pd.merge(data_BC,f,how='left')
data_BC = pd.merge(data_BC,g,how='left')
data_BC = pd.merge(data_BC,h,how='left')
data_BC = pd.merge(data_BC,i,how='left')

test_BC = pd.merge(test_BC,a,how='left')
test_BC = pd.merge(test_BC,b,how='left')
test_BC = pd.merge(test_BC,c,how='left')
# test_BC = pd.merge(test_BC,d,how='left')
# test_BC = pd.merge(test_BC,e,how='left')
# test_BC = pd.merge(test_BC,f,how='left')
test_BC = pd.merge(test_BC,g,how='left')
test_BC = pd.merge(test_BC,h,how='left')
test_BC = pd.merge(test_BC,i,how='left')



a = data_RD.groupby(['corporation','month'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'month_price'})
b = data_RD.groupby(['corporation','week'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'week_price'})
c = data_RD.groupby(['corporation','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price'})
# d = data_RD.groupby(['corporation','year','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price_year'})
# e = data_RD.groupby(['corporation','season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'season_price'})
# f = data_RD.groupby(['season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'t_season_price'})

g = data_RD.groupby(['corporation','month'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'month_supply'})
h = data_RD.groupby(['corporation','season'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'season_supply'})
i = data_RD.groupby(['corporation','dayofweek'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'dayofweek_supply'})



data_RD = pd.merge(data_RD,a,how='left')
data_RD = pd.merge(data_RD,b,how='left')
data_RD = pd.merge(data_RD,c,how='left')
# data_RD = pd.merge(data_RD,d,how='left')
# data_RD = pd.merge(data_RD,e,how='left')
# data_RD = pd.merge(data_RD,f,how='left')
data_RD = pd.merge(data_RD,g,how='left')
data_RD = pd.merge(data_RD,h,how='left')
data_RD = pd.merge(data_RD,i,how='left')



test_RD = pd.merge(test_RD,a,how='left')
test_RD = pd.merge(test_RD,b,how='left')
test_RD = pd.merge(test_RD,c,how='left')
# test_RD = pd.merge(test_RD,d,how='left')
# test_RD = pd.merge(test_RD,e,how='left')
# test_RD = pd.merge(test_RD,f,how='left')
test_RD = pd.merge(test_RD,g,how='left')
test_RD = pd.merge(test_RD,h,how='left')
test_RD = pd.merge(test_RD,i,how='left')



a = data_CR.groupby(['corporation','month'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'month_price'})
b = data_CR.groupby(['corporation','week'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'week_price'})
c = data_CR.groupby(['corporation','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price'})
# d = data_CR.groupby(['corporation','year','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price_year'})
# e = data_CR.groupby(['corporation','season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'season_price'})
# f = data_CR.groupby(['season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'t_season_price'})

g = data_CR.groupby(['corporation','month'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'month_supply'})
h = data_CR.groupby(['corporation','season'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'season_supply'})
i = data_CR.groupby(['corporation','dayofweek'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'dayofweek_supply'})

data_CR = pd.merge(data_CR,a,how='left')
data_CR = pd.merge(data_CR,b,how='left')
data_CR = pd.merge(data_CR,c,how='left')
# data_CR = pd.merge(data_CR,d,how='left')
# data_CR = pd.merge(data_CR,e,how='left')
# data_CR = pd.merge(data_CR,f,how='left')
data_CR = pd.merge(data_CR,g,how='left')
data_CR = pd.merge(data_CR,h,how='left')
data_CR = pd.merge(data_CR,i,how='left')


test_CR = pd.merge(test_CR,a,how='left')
test_CR = pd.merge(test_CR,b,how='left')
test_CR = pd.merge(test_CR,c,how='left')
# test_CR = pd.merge(test_CR,d,how='left')
# test_CR = pd.merge(test_CR,e,how='left')
# test_CR = pd.merge(test_CR,f,how='left')
test_CR = pd.merge(test_CR,g,how='left')
test_CR = pd.merge(test_CR,h,how='left')
test_CR = pd.merge(test_CR,i,how='left')


a = data_CB.groupby(['corporation','month'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'month_price'})
b = data_CB.groupby(['corporation','week'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'week_price'})
c = data_CB.groupby(['corporation','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price'})
# d = data_CB.groupby(['corporation','year','dayofweek'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'dayofweek_price_year'})
# e = data_CB.groupby(['corporation','season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'season_price'})
# f = data_CB.groupby(['season'])[['price(원/kg)']].mean().reset_index().rename(columns={'price(원/kg)':'t_season_price'})

g = data_CB.groupby(['corporation','month'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'month_supply'})
h = data_CB.groupby(['corporation','season'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'season_supply'})
i = data_CB.groupby(['corporation','dayofweek'])[['supply(kg)']].mean().reset_index().rename(columns={'supply(kg)':'dayofweek_supply'})



data_CB = pd.merge(data_CB,a,how='left')
data_CB = pd.merge(data_CB,b,how='left')
data_CB = pd.merge(data_CB,c,how='left')
# data_CB = pd.merge(data_CB,d,how='left')
# data_CB = pd.merge(data_CB,e,how='left')
# data_CB = pd.merge(data_CB,f,how='left')
data_CB = pd.merge(data_CB,g,how='left')
data_CB = pd.merge(data_CB,h,how='left')
data_CB = pd.merge(data_CB,i,how='left')



test_CB = pd.merge(test_CB,a,how='left')
test_CB = pd.merge(test_CB,b,how='left')
test_CB = pd.merge(test_CB,c,how='left')
# test_CB = pd.merge(test_CB,d,how='left')
# test_CB = pd.merge(test_CB,e,how='left')
# test_CB = pd.merge(test_CB,f,how='left')
test_CB = pd.merge(test_CB,g,how='left')
test_CB = pd.merge(test_CB,h,how='left')
test_CB = pd.merge(test_CB,i,how='left')



data_TG = data_TG.drop(['ID','supply(kg)','timestamp'],axis=1)
data_RD = data_RD.drop(['ID','supply(kg)','timestamp'],axis=1)
data_BC = data_BC.drop(['ID','supply(kg)','timestamp'],axis=1)
data_CB = data_CB.drop(['ID','supply(kg)','timestamp'],axis=1)
data_CR = data_CR.drop(['ID','supply(kg)','timestamp'],axis=1)

test_TGid = test_TG['ID']
test_RDid = test_RD['ID']
test_BCid = test_BC['ID']
test_CBid = test_CB['ID']
test_CRid = test_CR['ID']

test_TG = test_TG.drop(['ID','timestamp'],axis=1)
test_RD = test_RD.drop(['ID','timestamp'],axis=1)
test_BC = test_BC.drop(['ID','timestamp'],axis=1)
test_CB = test_CB.drop(['ID','timestamp'],axis=1)
test_CR = test_CR.drop(['ID','timestamp'],axis=1)

data_TG = pd.get_dummies(data_TG,columns=['corporation'],drop_first=True)
data_RD = pd.get_dummies(data_RD,columns=['corporation'],drop_first=True)
data_BC = pd.get_dummies(data_BC,columns=['corporation'],drop_first=True)
data_CB = pd.get_dummies(data_CB,columns=['corporation'],drop_first=True)
data_CR = pd.get_dummies(data_CR,columns=['corporation'],drop_first=True)

test_TG = pd.get_dummies(test_TG,columns=['corporation'],drop_first=True)
test_RD = pd.get_dummies(test_RD,columns=['corporation'],drop_first=True)
test_BC = pd.get_dummies(test_BC,columns=['corporation'],drop_first=True)
test_CB = pd.get_dummies(test_CB,columns=['corporation'],drop_first=True)
test_CR = pd.get_dummies(test_CR,columns=['corporation'],drop_first=True)

In [141]:
x = data_TG.drop('price(원/kg)',axis=1)
y = data_TG['price(원/kg)']

x['week'] = x['week'].astype(int)

model_rf = CatBoostRegressor(random_state=42)
model_rf.fit(x, y,verbose=0)
y_pred_all = model_rf.predict(test_TG)
y_pred_all = np.where(y_pred_all < 0,0,y_pred_all)

pred_tg = pd.DataFrame(columns=['ID'],data=test_TGid)
pred_tg['answer'] = y_pred_all

In [142]:
x = data_BC.drop('price(원/kg)',axis=1)
y = data_BC['price(원/kg)']

x['week'] = x['week'].astype(int)

model_rf = CatBoostRegressor(random_state=42)
model_rf.fit(x, y,verbose=0)
y_pred_all = model_rf.predict(test_BC)
y_pred_all = np.where(y_pred_all < 0,0,y_pred_all)

pred_BC = pd.DataFrame(columns=['ID'],data=test_BCid)
pred_BC['answer'] = y_pred_all

In [143]:
x = data_RD.drop('price(원/kg)',axis=1)
y = data_RD['price(원/kg)']

x['week'] = x['week'].astype(int)

model_rf = CatBoostRegressor(random_state=42)
model_rf.fit(x, y,verbose=0)
y_pred_all = model_rf.predict(test_RD)
y_pred_all = np.where(y_pred_all < 0,0,y_pred_all)

pred_RD = pd.DataFrame(columns=['ID'],data=test_RDid)
pred_RD['answer'] = y_pred_all

In [144]:
x = data_CB.drop('price(원/kg)',axis=1)
y = data_CB['price(원/kg)']

x['week'] = x['week'].astype(int)

model_rf = CatBoostRegressor(random_state=42)
model_rf.fit(x, y,verbose=0)
y_pred_all = model_rf.predict(test_CB)
y_pred_all = np.where(y_pred_all < 0,0,y_pred_all)

pred_CB = pd.DataFrame(columns=['ID'],data=test_CBid)
pred_CB['answer'] = y_pred_all

In [145]:
x = data_CR.drop('price(원/kg)',axis=1)
y = data_CR['price(원/kg)']

x['week'] = x['week'].astype(int)

model_rf = CatBoostRegressor(random_state=42)
model_rf.fit(x, y,verbose=0)
y_pred_all = model_rf.predict(test_CR)
y_pred_all = np.where(y_pred_all < 0,0,y_pred_all)

pred_CR = pd.DataFrame(columns=['ID'],data=test_CRid)
pred_CR['answer'] = y_pred_all

pred = pd.concat([pred_tg,pred_BC,pred_RD,pred_CB,pred_CR],axis=0)

In [146]:
sub = pd.read_csv('sample_submission.csv')
sub = sub.drop('answer',axis=1)
sub = pd.merge(sub,pred,how='left')
sub

,ID,answer
0,TG_A_J_20230304,3081.048377
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3482.105788
3,TG_A_J_20230307,3360.573069
4,TG_A_J_20230308,3479.369061
...,...,...
1087,RD_F_J_20230327,522.947712
1088,RD_F_J_20230328,524.143446
1089,RD_F_J_20230329,496.292526
1090,RD_F_J_20230330,482.561402
